In this notebook I shall load and clean the land use data. I shall then ensure the census data and land use data is harmonized as a number of Local Authorities changed in the time between the two datasets were published. I will then merge the dataframes and use the combined data to create calculate population density per Local Authority.   

In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import re

In [2]:
#load land use data
land_use_df = pd.read_excel('Data/Land_Use_England_2018.xlsx', sheet_name='P400b', skiprows=lambda x: x in [0,1]) 
col_names = land_use_df.columns.values.tolist()
land_use_df.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Community Service,Defence Buildings,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,"Forest, open land and water",Outdoor recreation,Residential Gardens,Undeveloped land,Unnamed: 41,Non-developed Use,Unnamed: 43,Vacant,Unnamed: 45,Unnamed: 46
0,NaN,NaN,NaN,Community Buildings,Leisure and recreational buildings,Total,Total,Industry,Offices,Retail,...,Total,Total,Total,Total,NaN,Total,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,England,NaN,93491.087766,584.034064,94075.121831,124.920265,18657.980659,7129.774265,11277.09771,...,2779193.076341,289759.684248,633383.384814,104139.452472,NaN,12128884.31023,NaN,22979.610234,NaN,1.325743e+07
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ONS Code,Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,E12000001,North East,NaN,5305.544389,43.087363,5348.631753,3.416098,1825.777728,374.894628,674.181709,...,341386.544687,14086.319795,22980.450424,6934.839141,NaN,805709.598981,NaN,1321.377965,NaN,8.673976e+05
6,E12000002,North West,NaN,12046.474266,72.643596,12119.117862,4.093185,3042.419359,1179.570717,1859.270599,...,535184.713732,35691.281031,68676.203081,14071.398648,NaN,1348907.426965,NaN,2186.787003,NaN,1.485416e+06
7,E12000003,Yorkshire and The Humber,NaN,9300.505675,62.84281,9363.348485,14.774515,3445.211559,522.541923,1066.332916,...,395853.079297,25838.666962,57981.233344,12256.70608,NaN,1435214.499328,NaN,1906.812577,NaN,1.555162e+06
8,E12000004,East Midlands,NaN,8557.350873,47.276395,8604.627269,5.641154,2101.052211,826.020658,1171.331674,...,196670.047709,27311.589053,62674.031408,10083.289717,NaN,1456146.450134,NaN,2854.911589,NaN,1.580851e+06
9,E12000005,West Midlands,NaN,10434.317971,91.525782,10525.843753,8.474661,2064.183642,727.134699,1348.286577,...,162838.410412,31466.023778,70111.705214,10855.571891,NaN,1181989.690094,NaN,2311.611821,NaN,1.300313e+06


In [3]:
#Reconfigure land use dataframe and remove unnecessary columns
land_use_df = land_use_df[['Unnamed: 0', 'Unnamed: 1', 'Developed Use ', 'Non-developed Use', 'Unnamed: 46']]
land_use_df.rename(columns={'Unnamed: 0': 'ONS Code', 'Unnamed: 1': 'Local Authority Name', 'Unnamed: 46': 'Total Land Area' }, inplace=True)
land_use_df.drop([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16], axis=0, inplace = True)
print(land_use_df.shape)
land_use_df.head()

(326, 5)


,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area
17,E07000223,Adur,941.534725,3418.435055,4364.505691
18,E07000026,Allerdale,4844.924035,127158.413179,132054.795869
19,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279
20,E07000224,Arun,2791.129278,19613.024334,22450.444783
21,E07000170,Ashfield,2028.578319,8881.797678,10955.544524


In [4]:
#Remove white space from column name
land_use_df.rename(columns={"Developed Use ": "Developed Use"}, inplace = True)

The two datasets I am using have some differences in the Local Authorities included. The census data is from 2021 whereas the land use data was published in 2018. The census data included a list of  7 Local Authorities which had been newly created in 2020 combining several previous Local Authorities. 

In order to merge the two datasets and perform meaningful data analysis I would like to include these new Local Authorities so I am going to aggregate and sum the land use data for the Local Authorities which no longer exist to create rows for the Local Authorities created in 2020 so that the land use data and the census data match.  

In [5]:
def combine_las_to_remove(remove_list, land_use_df):
    """Creates a new dataframe with data for LAs that have been combined into one new LA."""
    #df = pd.DataFrame(columns=['ONS Code','Local Authority Name','Developed Use','Non-developed Use','Total Land Area'])
    empty_list = []
    for index, row in land_use_df.iterrows():
        if row['Local Authority Name'] in remove_list:
            md = land_use_df.loc[index, :].values.flatten().tolist()
            empty_list.append(md)
    add_df = pd.DataFrame(empty_list, columns=['ONS Code','Local Authority Name','Developed Use','Non-developed Use','Total Land Area'])
    print(add_df)
    return add_df


def sum_las_to_remove(add_df, new_la_code, new_la_name):
    """Takes df and totals all numeric columns to create new row with data for new combined LA"""
    add_df.loc['Column_Total']= add_df.sum(numeric_only=True, axis=0)
    add_df.loc['Column_Total', 'Local Authority Name'] = new_la_name
    add_df.loc['Column_Total', 'ONS Code'] = new_la_code
    return add_df


def add_new_la_delete_abolished_las(land_use_df, sum_df, remove_list):
    """Adds new  combined LA row to land_use_df and removes rows which refer to those former
    LAs which no longer exist"""
    land_use_df = pd.concat([land_use_df, sum_df.tail(1)], axis=0)
    land_use_df = land_use_df[land_use_df['Local Authority Name'].isin(remove_list) == False]
    return land_use_df
    


In [6]:
#Create new LA North Northamptonshire
northamptonshire_remove = ['Corby', 'East Northamptonshire', 'Kettering', 'Wellingborough']    
nn = sum_las_to_remove(combine_las_to_remove(northamptonshire_remove, land_use_df), 'E06000061', 'North Northamptonshire')
 
land_use_df = add_new_la_delete_abolished_las(land_use_df, nn, northamptonshire_remove)


    ONS Code   Local Authority Name  Developed Use  Non-developed Use  \
0  E07000150                  Corby    1465.991360        6377.268316   
1  E07000152  East Northamptonshire    4791.998684       46134.578032   
2  E07000153              Kettering    2163.510188       21092.661844   
3  E07000156         Wellingborough    1831.957328       14397.351187   

   Total Land Area  
0      8027.939668  
1     50978.586697  
2     23348.916855  
3     16303.622392  


In [7]:
#Create new LA West Northamptonshire
west_northamptonshire_remove = ['Daventry', 'Northampton', 'South Northamptonshire']
wn = sum_las_to_remove(combine_las_to_remove(west_northamptonshire_remove, land_use_df), 'E06000062', 'West Northamptonshire')
 
land_use_df = add_new_la_delete_abolished_las(land_use_df, wn, west_northamptonshire_remove)
land_use_df

    ONS Code    Local Authority Name  Developed Use  Non-developed Use  \
0  E07000151                Daventry    3457.307159       62792.502320   
1  E07000154             Northampton    3064.873298        4952.512582   
2  E07000155  South Northamptonshire    3461.975891       59819.154564   

   Total Land Area  
0     66559.824011  
1      8076.167194  
2     63399.917721  


,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area
17,E07000223,Adur,941.534725,3418.435055,4364.505691
18,E07000026,Allerdale,4844.924035,127158.413179,132054.795869
19,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279
20,E07000224,Arun,2791.129278,19613.024334,22450.444783
21,E07000170,Ashfield,2028.578319,8881.797678,10955.544524
...,...,...,...,...,...
340,E07000128,Wyre,2571.244346,29877.321859,32495.519662
341,E07000239,Wyre Forest,1937.962717,17580.747614,19534.968482
342,E06000014,York,3664.585248,23503.199857,27198.689875
Column_Total,E06000061,North Northamptonshire,10253.457561,88001.859379,98659.065612


In [8]:
#Create new LA East Suffolk
east_suffolk_remove = ['Suffolk Coastal', 'Waveney']

es = sum_las_to_remove(combine_las_to_remove(east_suffolk_remove, land_use_df), 'E07000244', 'East Suffolk')
 
land_use_df = add_new_la_delete_abolished_las(land_use_df, es, east_suffolk_remove)
land_use_df

    ONS Code Local Authority Name  Developed Use  Non-developed Use  \
0  E07000205      Suffolk Coastal    5221.292686       86760.638200   
1  E07000206              Waveney    2903.206311       34247.957062   

   Total Land Area  
0     92032.120098  
1     37186.018738  


,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area
17,E07000223,Adur,941.534725,3418.435055,4364.505691
18,E07000026,Allerdale,4844.924035,127158.413179,132054.795869
19,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279
20,E07000224,Arun,2791.129278,19613.024334,22450.444783
21,E07000170,Ashfield,2028.578319,8881.797678,10955.544524
...,...,...,...,...,...
341,E07000239,Wyre Forest,1937.962717,17580.747614,19534.968482
342,E06000014,York,3664.585248,23503.199857,27198.689875
Column_Total,E06000061,North Northamptonshire,10253.457561,88001.859379,98659.065612
Column_Total,E06000062,West Northamptonshire,9984.156347,127564.169465,138035.908925


In [9]:
#Create new LA West Suffolk
west_suffolk_remove = ['Forest Heath', 'St Edmundsbury']

ws = sum_las_to_remove(combine_las_to_remove(west_suffolk_remove, land_use_df), 'E07000245', 'West Suffolk')
 
land_use_df = add_new_la_delete_abolished_las(land_use_df, ws, west_suffolk_remove)
land_use_df



    ONS Code Local Authority Name  Developed Use  Non-developed Use  \
0  E07000201         Forest Heath    3314.602865       34398.542398   
1  E07000204       St Edmundsbury    4225.520842       61359.313296   

   Total Land Area  
0     37753.030274  
1     65695.866382  


,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area
17,E07000223,Adur,941.534725,3418.435055,4364.505691
18,E07000026,Allerdale,4844.924035,127158.413179,132054.795869
19,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279
20,E07000224,Arun,2791.129278,19613.024334,22450.444783
21,E07000170,Ashfield,2028.578319,8881.797678,10955.544524
...,...,...,...,...,...
342,E06000014,York,3664.585248,23503.199857,27198.689875
Column_Total,E06000061,North Northamptonshire,10253.457561,88001.859379,98659.065612
Column_Total,E06000062,West Northamptonshire,9984.156347,127564.169465,138035.908925
Column_Total,E07000244,East Suffolk,8124.498998,121008.595262,129218.138837


In [10]:
#Create new LA Buckinghamshire
buckinghamshire_remove = ['Aylesbury Vale', 'Chiltern', 'South Bucks', 'Wycombe']

bu = sum_las_to_remove(combine_las_to_remove(buckinghamshire_remove, land_use_df), 'E06000060', 'Buckinghamshire')
 
land_use_df = add_new_la_delete_abolished_las(land_use_df, bu, buckinghamshire_remove)
land_use_df



    ONS Code Local Authority Name  Developed Use  Non-developed Use  \
0  E07000004       Aylesbury Vale    5614.533011       84446.918097   
1  E07000005             Chiltern    1762.424950       17854.434945   
2  E07000006          South Bucks    1947.908308       12147.920232   
3  E07000007              Wycombe    3035.642362       29382.736449   

   Total Land Area  
0     90271.305526  
1     19634.287278  
2     14126.190591  
3     32456.862300  


,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area
17,E07000223,Adur,941.534725,3418.435055,4364.505691
18,E07000026,Allerdale,4844.924035,127158.413179,132054.795869
19,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279
20,E07000224,Arun,2791.129278,19613.024334,22450.444783
21,E07000170,Ashfield,2028.578319,8881.797678,10955.544524
...,...,...,...,...,...
Column_Total,E06000061,North Northamptonshire,10253.457561,88001.859379,98659.065612
Column_Total,E06000062,West Northamptonshire,9984.156347,127564.169465,138035.908925
Column_Total,E07000244,East Suffolk,8124.498998,121008.595262,129218.138837
Column_Total,E07000245,West Suffolk,7540.123708,95757.855695,103448.896657


In [11]:
#Create new LA Bournemouth, Christchurch and Poole
bournemouth_remove = ['Christchurch', 'Bournemouth', 'Poole']
bo = sum_las_to_remove(combine_las_to_remove(bournemouth_remove, land_use_df), 'E06000058', 'Bournemouth, Christchurch and Poole')
 
land_use_df = add_new_la_delete_abolished_las(land_use_df, bo, bournemouth_remove)
land_use_df


    ONS Code Local Authority Name  Developed Use  Non-developed Use  \
0  E06000028          Bournemouth    1811.915546        2918.119748   
1  E07000048         Christchurch    1234.350128        3931.935215   
2  E06000029                Poole    2128.148876        5325.401353   

   Total Land Area  
0      4734.429178  
1      5169.241925  
2      7475.605457  


,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area
17,E07000223,Adur,941.534725,3418.435055,4364.505691
18,E07000026,Allerdale,4844.924035,127158.413179,132054.795869
19,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279
20,E07000224,Arun,2791.129278,19613.024334,22450.444783
21,E07000170,Ashfield,2028.578319,8881.797678,10955.544524
...,...,...,...,...,...
Column_Total,E06000062,West Northamptonshire,9984.156347,127564.169465,138035.908925
Column_Total,E07000244,East Suffolk,8124.498998,121008.595262,129218.138837
Column_Total,E07000245,West Suffolk,7540.123708,95757.855695,103448.896657
Column_Total,E06000060,Buckinghamshire,12360.508631,143832.009723,156488.645695


In [12]:
#Create new LA Dorset
dorset_remove = ['East Dorset', 'North Dorset', 'Purbeck', 'West Dorset', 'Weymouth and Portland']
do = sum_las_to_remove(combine_las_to_remove(dorset_remove, land_use_df), 'E06000059', 'Dorset')
 
land_use_df = add_new_la_delete_abolished_las(land_use_df, do, dorset_remove)
land_use_df



    ONS Code   Local Authority Name  Developed Use  Non-developed Use  \
0  E07000049            East Dorset    2568.760317       32834.893333   
1  E07000050           North Dorset    2871.859823       58023.678570   
2  E07000051                Purbeck    2505.263248       40215.201047   
3  E07000052            West Dorset    4808.103449      103802.947421   
4  E07000053  Weymouth and Portland     953.383341        3334.330291   

   Total Land Area  
0     35424.549588  
1     60905.907559  
2     42732.640241  
3    108674.947336  
4      4298.340078  


,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area
17,E07000223,Adur,941.534725,3418.435055,4364.505691
18,E07000026,Allerdale,4844.924035,127158.413179,132054.795869
19,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279
20,E07000224,Arun,2791.129278,19613.024334,22450.444783
21,E07000170,Ashfield,2028.578319,8881.797678,10955.544524
...,...,...,...,...,...
Column_Total,E07000244,East Suffolk,8124.498998,121008.595262,129218.138837
Column_Total,E07000245,West Suffolk,7540.123708,95757.855695,103448.896657
Column_Total,E06000060,Buckinghamshire,12360.508631,143832.009723,156488.645695
Column_Total,E06000058,"Bournemouth, Christchurch and Poole",5174.41455,12175.456316,17379.276560


In [13]:
#Create new LA Somerset West and Taunton
somerset_remove = ['Taunton Deane', 'West Somerset']
so = sum_las_to_remove(combine_las_to_remove(somerset_remove, land_use_df), 'E07000246', 'Somerset West and Taunton')
 
land_use_df = add_new_la_delete_abolished_las(land_use_df, so, somerset_remove)
land_use_df = land_use_df.reset_index()
land_use_df = land_use_df.drop(['index'],  axis=1)
land_use_df


    ONS Code Local Authority Name  Developed Use  Non-developed Use  \
0  E07000190        Taunton Deane    3396.448087       42788.369460   
1  E07000191        West Somerset    2457.754874       72294.996021   

   Total Land Area  
0     46250.105992  
1     74755.219251  


,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area
0,E07000223,Adur,941.534725,3418.435055,4364.505691
1,E07000026,Allerdale,4844.924035,127158.413179,132054.795869
2,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279
3,E07000224,Arun,2791.129278,19613.024334,22450.444783
4,E07000170,Ashfield,2028.578319,8881.797678,10955.544524
...,...,...,...,...,...
304,E07000245,West Suffolk,7540.123708,95757.855695,103448.896657
305,E06000060,Buckinghamshire,12360.508631,143832.009723,156488.645695
306,E06000058,"Bournemouth, Christchurch and Poole",5174.41455,12175.456316,17379.276560
307,E06000059,Dorset,13707.370179,238211.050661,252036.384802


Now I want to calculate the percentage of developed land in each Local Authority as I am intending to use this as one of the measure of how urbanised a Local Authority is.   

In [14]:
#Calculate percentage of deveopled land in each Local Authority
land_use_df['% Developed Land'] = land_use_df['Developed Use']/land_use_df['Total Land Area']*100
decimals = 2    
land_use_df['% Developed Land'] = land_use_df['% Developed Land'].apply(lambda x: round(x, decimals))
land_use_df

,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area,% Developed Land
0,E07000223,Adur,941.534725,3418.435055,4364.505691,21.57
1,E07000026,Allerdale,4844.924035,127158.413179,132054.795869,3.67
2,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279,9.82
3,E07000224,Arun,2791.129278,19613.024334,22450.444783,12.43
4,E07000170,Ashfield,2028.578319,8881.797678,10955.544524,18.52
...,...,...,...,...,...,...
304,E07000245,West Suffolk,7540.123708,95757.855695,103448.896657,7.29
305,E06000060,Buckinghamshire,12360.508631,143832.009723,156488.645695,7.90
306,E06000058,"Bournemouth, Christchurch and Poole",5174.41455,12175.456316,17379.276560,29.77
307,E06000059,Dorset,13707.370179,238211.050661,252036.384802,5.44


Now I shall load the age_group_proportions_census data I produced and saved earlier.

In [15]:
census_age_proportion_df = pd.read_csv('age_group_proportions_census_df.csv', index_col=0)
census_age_proportion_df

,Area name,Area code,Region,Proportion pop aged 0-19,Proportion pop aged 20-64,Proportion pop aged 65+,Proportion pop aged 0-34,All persons
0,County Durham,E06000047,North East,21.796591,56.904808,21.356062,39.590117,522100
1,Darlington,E06000005,North East,22.541744,56.957328,20.686456,39.888683,107800
2,Hartlepool,E06000001,North East,23.401950,56.663055,19.718310,41.170098,92300
3,Middlesbrough,E06000002,North East,25.573315,57.748436,16.817234,46.977067,143900
4,Northumberland,E06000057,North East,19.962570,54.585153,25.452277,34.591391,320600
...,...,...,...,...,...,...,...,...
326,Caerphilly,W06000018,Wales,22.797044,57.248437,20.011370,40.648096,175900
327,Blaenau Gwent,W06000019,Wales,21.375187,58.295964,20.328849,40.209268,66900
328,Torfaen,W06000020,Wales,22.535211,56.446371,20.801733,40.628386,92300
329,Monmouthshire,W06000021,Wales,19.677419,54.623656,25.913978,33.763441,93000


The census data includes Wales whereas the land use data does not. So to combine the two dataframes I shall drop the rows with Welsh local authorities as I have no land use data for them.

In [16]:
country = ['Wales']

#Drop any rows from dataframe which include Wales in the region column
census_age_proportion_df = census_age_proportion_df[census_age_proportion_df['Region'].isin(country) == False]
census_age_proportion_df

,Area name,Area code,Region,Proportion pop aged 0-19,Proportion pop aged 20-64,Proportion pop aged 65+,Proportion pop aged 0-34,All persons
0,County Durham,E06000047,North East,21.796591,56.904808,21.356062,39.590117,522100
1,Darlington,E06000005,North East,22.541744,56.957328,20.686456,39.888683,107800
2,Hartlepool,E06000001,North East,23.401950,56.663055,19.718310,41.170098,92300
3,Middlesbrough,E06000002,North East,25.573315,57.748436,16.817234,46.977067,143900
4,Northumberland,E06000057,North East,19.962570,54.585153,25.452277,34.591391,320600
...,...,...,...,...,...,...,...,...
304,Tewkesbury,E07000083,South West,22.233930,55.321391,22.233930,38.566913,94900
305,Mendip,E07000187,South West,21.705426,54.521964,23.772610,36.434109,116100
306,Sedgemoor,E07000188,South West,21.212121,54.864434,23.763955,37.559809,125400
307,Somerset West and Taunton,E07000246,South West,20.648030,53.748412,25.603558,36.213469,157400


Now I shall use sets to check whether the Local Authorities in both my census dataframe and my land use dataframe correspond exactly with one another. 

In [17]:
census_list = census_age_proportion_df['Area name'].tolist()
land_use_list = land_use_df['Local Authority Name'].tolist()

#Use sets to check two lists are identical
c = set(census_list)-set(land_use_list)
d = set(land_use_list)-set(census_list)
e = []
e.append(d)
e.append(c)
e

[{"King's Lynn and West Norfolk"}, {'King’s Lynn and West Norfolk'}]

There is one discrepancy between the two dataframes, one Local Authority has a different type of apostrophe in the two dataframes so I shall amend this so they are both the same and then check that has been done.

In [18]:
#Amend apostophe in census data
kl = census_age_proportion_df[census_age_proportion_df == 'King’s Lynn and West Norfolk'].stack().index.tolist()

census_age_proportion_df.at[kl[0]] = "King's Lynn and West Norfolk"

In [19]:
#Use sets to check two lists are identical
census_list = census_age_proportion_df['Area name'].tolist()
c = set(census_list)-set(land_use_list)
d = set(land_use_list)-set(census_list)
e = []
e.append(d)
e.append(c)
e

[set(), set()]

That has worked. The two dataframes I wish to combine have different column names for the same information so I will amend the census dataframe so it has the same column names as the land use dataframe. 

In [20]:
#Chnage column names in census data so matches land use data
census_age_proportion_df = census_age_proportion_df.rename(columns={'Area name': 'Local Authority Name', 'Area code': 'ONS Code'})
census_age_proportion_df.head()

,Local Authority Name,ONS Code,Region,Proportion pop aged 0-19,Proportion pop aged 20-64,Proportion pop aged 65+,Proportion pop aged 0-34,All persons
0,County Durham,E06000047,North East,21.796591,56.904808,21.356062,39.590117,522100
1,Darlington,E06000005,North East,22.541744,56.957328,20.686456,39.888683,107800
2,Hartlepool,E06000001,North East,23.401950,56.663055,19.718310,41.170098,92300
3,Middlesbrough,E06000002,North East,25.573315,57.748436,16.817234,46.977067,143900
4,Northumberland,E06000057,North East,19.962570,54.585153,25.452277,34.591391,320600


Now I will combine the two dataframes merging them with a left outer join on the ONS code and the Local Authority Name .  

In [21]:
combined_df = land_use_df.merge(census_age_proportion_df, how = 'outer', on = ['ONS Code', 'Local Authority Name'])
combined_df.head()

,ONS Code,Local Authority Name,Developed Use,Non-developed Use,Total Land Area,% Developed Land,Region,Proportion pop aged 0-19,Proportion pop aged 20-64,Proportion pop aged 65+,Proportion pop aged 0-34,All persons
0,E07000223,Adur,941.534725,3418.435055,4364.505691,21.57,South East,21.860465,54.418605,23.720930,36.279070,64500
1,E07000026,Allerdale,4844.924035,127158.413179,132054.795869,3.67,North West,20.083247,55.254943,24.869927,35.379813,96100
2,E07000032,Amber Valley,2606.772331,23861.418671,26543.663279,9.82,East Midlands,20.443740,56.973059,22.503962,36.846276,126200
3,E07000224,Arun,2791.129278,19613.024334,22450.444783,12.43,South East,19.114078,52.487864,28.398058,34.344660,164800
4,E07000170,Ashfield,2028.578319,8881.797678,10955.544524,18.52,East Midlands,22.723674,57.561362,19.635788,40.934283,126300


I will drop any columns I do not need and rename some of the columns.

In [22]:
combined_df = combined_df.drop(['Developed Use','Non-developed Use', 'Region', 'Proportion pop aged 0-19', 'Proportion pop aged 20-64', 'Proportion pop aged 65+' ], axis=1)
combined_df = combined_df.rename(columns={'Total Land Area': 'Total Land Area (hectares)', 'Area code': 'ONS Code'})
combined_df.head()

,ONS Code,Local Authority Name,Total Land Area (hectares),% Developed Land,Proportion pop aged 0-34,All persons
0,E07000223,Adur,4364.505691,21.57,36.279070,64500
1,E07000026,Allerdale,132054.795869,3.67,35.379813,96100
2,E07000032,Amber Valley,26543.663279,9.82,36.846276,126200
3,E07000224,Arun,22450.444783,12.43,34.344660,164800
4,E07000170,Ashfield,10955.544524,18.52,40.934283,126300


I would like to calculate the population density for each Local Authority and include this in the combined dataframe.

In [23]:
#Add new column with population density 
combined_df['Population Density (people per hectare)'] = combined_df['All persons']/ combined_df['Total Land Area (hectares)']
combined_df

,ONS Code,Local Authority Name,Total Land Area (hectares),% Developed Land,Proportion pop aged 0-34,All persons,Population Density (people per hectare)
0,E07000223,Adur,4364.505691,21.57,36.279070,64500,14.778306
1,E07000026,Allerdale,132054.795869,3.67,35.379813,96100,0.727728
2,E07000032,Amber Valley,26543.663279,9.82,36.846276,126200,4.754430
3,E07000224,Arun,22450.444783,12.43,34.344660,164800,7.340612
4,E07000170,Ashfield,10955.544524,18.52,40.934283,126300,11.528409
...,...,...,...,...,...,...,...
304,E07000245,West Suffolk,103448.896657,7.29,41.601780,179800,1.738056
305,E06000060,Buckinghamshire,156488.645695,7.90,40.462846,553100,3.534442
306,E06000058,"Bournemouth, Christchurch and Poole",17379.276560,29.77,40.144891,400300,23.033180
307,E06000059,Dorset,252036.384802,5.44,32.481560,379600,1.506132


I am happy with the combined dataframe and will save it.

In [24]:
combined_df.to_csv('combined_df.csv')